# SMART Maize farming advisory system using weather forecasts and AI
**Authors:** 
    - **JEMAL S. AHMED**
 

This notebook presents an automated advisory system for maize-growing smallholder farmers, designed to support decision-making across various crop growth stages, from pre-planting to harvesting. The system integrates weather forecast data retrieved from the EDACaP API and compares it against historical climate averages to identify anomalies in rainfall, temperature, humidity, and wind speed. These anomalies are then used to define tailored agronomic scenarios and recommendations for farmers. By leveraging machine learning models and integrating OpenAI's GPT-4, this system generates personalized, actionable advisories for farmers, helping them optimize planting dates, manage water and soil resources, protect crops from extreme weather, and improve yields. The visualizations in this notebook allow users to compare forecasted and historical weather patterns, offering deeper insights into how current conditions may impact farming decisions. Ultimately, this tool aims to enhance climate resilience and agronomic practices for smallholder farmers in Ethiopia.

In [5]:
import openai
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# Set OpenAI API key
openai.api_key = 'xxxxxxxxxxxxxxxxxxx'

In [6]:
# Fetch forecast data from Ethiopian EDACaP API
def fetch_weather_forecast(api_url):
    response = requests.get(api_url)
    return response.json()['data'] if response.status_code == 200 else None

# Calculate anomalies using location specific data
def calculate_anomalies(forecast_data, historical_averages):
    forecast_df = pd.DataFrame(forecast_data)
    forecast_df['datetime'] = pd.to_datetime(forecast_df['datetime'])
    forecast_df['month_day'] = forecast_df['datetime'].dt.strftime('%m-%d')
    anomalies_df = pd.merge(forecast_df, historical_averages, on='month_day', how='left')
    anomalies_df['precip_anomaly'] = anomalies_df['precip'] - anomalies_df['avg_prec']
    anomalies_df['t_max_anomaly'] = anomalies_df['max_temp'] - anomalies_df['avg_t_max']
    anomalies_df['t_min_anomaly'] = anomalies_df['min_temp'] - anomalies_df['avg_t_min']
    anomalies_df['humidity_anomaly'] = anomalies_df['rh'] - anomalies_df['avg_rh']
    anomalies_df['wind_speed_anomaly'] = anomalies_df['wind_spd'] - anomalies_df['avg_wind_spd']
    return anomalies_df

# Define multiple agronomic management scenarios with multiple options for different crop stages
def define_scenarios(anomalies_df, stage):
    scenarios = []
    for _, row in anomalies_df.iterrows():
        precip_anomaly = row['precip_anomaly']
        t_max_anomaly = row['t_max_anomaly']
        t_min_anomaly = row['t_min_anomaly']
        humidity_anomaly = row['humidity_anomaly']
        wind_speed_anomaly = row['wind_speed_anomaly']
        
        # Pre-Planting Stage
        if stage == "pre-planting":
            if precip_anomaly > 5:
                scenarios.append({
                    "date": row['datetime'].date(),
                    "scenario": "Heavy Rainfall Expected",
                    "options": [
                        "Option 1: Delay planting to avoid waterlogged conditions.",
                        "Option 2: Apply drainage techniques before planting to prevent waterlogging.",
                        "Option 3: Use raised beds to avoid seed damage in waterlogged areas."
                    ],
                    "additional_info": f"Wind speed is {row['wind_spd']} m/s. Strong wind might affect seed distribution if not protected."
                })
            else:
                scenarios.append({
                    "date": row['datetime'].date(),
                    "scenario": "Normal to Dry Conditions",
                    "options": [
                        "Option 1: Proceed with planting if soil moisture is sufficient.",
                        "Option 2: Consider light irrigation to ensure soil moisture is adequate.",
                        "Option 3: Monitor wind speed and humidity for ideal conditions."
                    ],
                    "additional_info": f"Humidity anomaly: {humidity_anomaly:.2f}. Ensure moisture retention with mulching."
                })

        # Planting Stage
        elif stage == "planting":
            if precip_anomaly > 5:
                scenarios.append({
                    "date": row['datetime'].date(),
                    "scenario": "Heavy Rainfall Expected",
                    "options": [
                        "Option 1: Avoid planting to prevent seed washout.",
                        "Option 2: Use mulching and soil protection to reduce erosion during planting.",
                        "Option 3: Delay planting until the soil stabilizes after heavy rainfall."
                    ],
                    "additional_info": f"Wind speed anomaly: {wind_speed_anomaly:.2f}. Protect young seeds from strong winds."
                })
            elif precip_anomaly < -2:
                scenarios.append({
                    "date": row['datetime'].date(),
                    "scenario": "Insufficient Rainfall",
                    "options": [
                        "Option 1: Irrigate soil before planting to ensure adequate moisture for germination.",
                        "Option 2: Delay planting until rainfall increases or apply supplemental irrigation.",
                        "Option 3: Use water-saving planting methods such as direct seeding."
                    ],
                    "additional_info": f"Low humidity ({row['rh']}%) and low rainfall may require more irrigation."
                })
        
        # Early Growth Stage
        elif stage == "early growth":
            if t_max_anomaly < -3:
                scenarios.append({
                    "date": row['datetime'].date(),
                    "scenario": "Cool Temperatures Expected",
                    "options": [
                        "Option 1: Avoid overwatering as cool temperatures reduce water uptake.",
                        "Option 2: Use mulching to maintain soil warmth and moisture.",
                        "Option 3: Delay fertilization until temperatures normalize to avoid nutrient leaching."
                    ],
                    "additional_info": f"Cool weather and low humidity ({row['rh']}%) may delay growth. Monitor for fungal diseases."
                })
            elif t_max_anomaly > 3:
                scenarios.append({
                    "date": row['datetime'].date(),
                    "scenario": "High Temperatures Expected",
                    "options": [
                        "Option 1: Increase irrigation to prevent heat stress on young plants.",
                        "Option 2: Apply mulch to retain soil moisture and reduce evaporation.",
                        "Option 3: Use shade netting or natural shading to protect seedlings from heat."
                    ],
                    "additional_info": f"High wind speeds ({row['wind_spd']} m/s) could dry out soil faster."
                })
        
        # Vegetative Growth Stage
        elif stage == "vegetative growth":
            if precip_anomaly < -3:
                scenarios.append({
                    "date": row['datetime'].date(),
                    "scenario": "Dry Conditions Expected",
                    "options": [
                        "Option 1: Apply regular irrigation to compensate for low rainfall.",
                        "Option 2: Monitor soil moisture and use organic mulch to conserve water.",
                        "Option 3: Apply water-saving irrigation techniques like drip irrigation."
                    ],
                    "additional_info": f"Humidity is low ({row['rh']}%), and wind speed is {row['wind_spd']} m/s, which could increase evaporation."
                })
            else:
                scenarios.append({
                    "date": row['datetime'].date(),
                    "scenario": "Favorable Growth Conditions",
                    "options": [
                        "Option 1: Maintain regular irrigation and fertilizer schedule.",
                        "Option 2: Monitor for pests due to favorable humidity levels ({row['rh']}%).",
                        "Option 3: Optimize nutrient application based on soil moisture and growth progress."
                    ]
                })
        
        # Reproductive Stage
        elif stage == "reproductive phase":
            if precip_anomaly > 5:
                scenarios.append({
                    "date": row['datetime'].date(),
                    "scenario": "Heavy Rainfall Expected During Tasseling",
                    "options": [
                        "Option 1: Ensure proper drainage to prevent water stress on the plant.",
                        "Option 2: Avoid pesticide or foliar fertilizer applications during heavy rain.",
                        "Option 3: Monitor plants for fungal disease due to increased moisture."
                    ],
                    "additional_info": f"High humidity ({row['rh']}%) may increase the risk of fungal diseases like rust or blight."
                })
            else:
                scenarios.append({
                    "date": row['datetime'].date(),
                    "scenario": "Normal Conditions for Tasseling and Silking",
                    "options": [
                        "Option 1: Maintain optimal water supply during grain filling.",
                        "Option 2: Continue with the normal fertilization and pest management schedule.",
                        "Option 3: Use foliar micronutrient application to boost grain filling."
                    ]
                })
        
        # Harvesting Stage
        elif stage == "harvesting":
            if precip_anomaly > 5:
                scenarios.append({
                    "date": row['datetime'].date(),
                    "scenario": "Rain Expected During Harvest",
                    "options": [
                        "Option 1: Delay harvesting to avoid grain spoilage due to moisture.",
                        "Option 2: Harvest early if dry conditions are expected after the rainfall.",
                        "Option 3: Plan for post-harvest drying facilities if rain cannot be avoided."
                    ],
                    "additional_info": f"High humidity and precipitation might delay drying processes."
                })
            else:
                scenarios.append({
                    "date": row['datetime'].date(),
                    "scenario": "Dry Conditions Expected During Harvest",
                    "options": [
                        "Option 1: Harvest under optimal dry conditions to avoid grain spoilage.",
                        "Option 2: Prepare storage facilities and ensure proper ventilation.",
                        "Option 3: Start drying harvested grain immediately to maintain quality."
                    ]
                })
    
    return scenarios



In [7]:
# Generate detailed advisory for smallholder farmer using ChatGPT
def generate_advisory_using_chatgpt(scenarios, stage):
    """
    Generate advisory content using ChatGPT based on the scenarios.
    """
    advisories = []
    for scenario in scenarios:
        options = "\n".join(scenario['options'])
        prompt = f"""
        Scenario: {scenario['scenario']} on {scenario['date']} during the {stage} stage of maize farming.
        Options:
        {options}
        
        Additional Info: {scenario.get('additional_info', 'None')}
        
        Please provide a detailed advisory on how the farmer can choose between the options and justify each action.
        """

        response = openai.Completion.create(
            engine="gpt-4",  # Using GPT-4 model
            prompt=prompt,
            max_tokens=300,
            temperature=0.7
        )

        advisories.append(response.choices[0].text.strip())
    
    return advisories

# ----------------------------------------------
# Visualize the forecast data and anomalies
def plot_forecast_vs_historical(anomalies_df, stage):
    """
    Plot forecasted vs historical data with anomalies highlighted.
    """
    plt.figure(figsize=(10, 6))
    
    sns.lineplot(x='datetime', y='precip', data=anomalies_df, label='Forecasted Rainfall', color='blue')
    sns.lineplot(x='datetime', y='avg_prec', data=anomalies_df, label='Historical Average Rainfall', color='orange')
    
    plt.fill_between(anomalies_df['datetime'], anomalies_df['avg_prec'], anomalies_df['precip'], 
                     where=(anomalies_df['precip'] > anomalies_df['avg_prec']), color='green', alpha=0.3, label='Above Normal')
    plt.fill_between(anomalies_df['datetime'], anomalies_df['avg_prec'], anomalies_df['precip'], 
                     where=(anomalies_df['precip'] < anomalies_df['avg_prec']), color='red', alpha=0.3, label='Below Normal')
    
    plt.title(f"Rainfall Forecast vs Historical Averages ({stage.capitalize()})")
    plt.xlabel('Date')
    plt.ylabel('Rainfall (mm)')
    plt.legend()
    plt.grid(True)
    plt.show()



In [11]:
# Load the historical daily climate data 
historical_data = pd.read_csv("EDACaP040706_daily.csv")
historical_data['month_day'] = historical_data['month'].astype(str).str.zfill(2) + '-' + historical_data['day'].astype(str).str.zfill(2)

historical_averages = historical_data.groupby('month_day').agg({
    'prec': 'mean',      # Average daily precipitation
    't_max': 'mean',     # Average daily max temperature
    't_min': 'mean'      # Average daily min temperature
}).reset_index()

historical_averages.columns = ['month_day', 'avg_prec', 'avg_t_max', 'avg_t_min']
historical_averages.head()

,month_day,avg_prec,avg_t_max,avg_t_min
0,01-01,0.05700,24.97225,10.24625
1,01-02,0.04975,25.12875,10.40100
2,01-03,0.49800,25.20550,10.57725
3,01-04,0.00000,25.06475,10.52650
4,01-05,0.06475,25.25850,10.36025


In [12]:
###### Automate the advisory generation process

# Define API endpoint for forecast data and load historical averages
api_url = "https://edacapapi.ethioagroclimate.net/forecast/daily?lat=7.9621&lon=39.1284"
historical_averages_file = "EDACaP040706_daily.csv"

# Load the historical daily climate data and calculate historical averages
historical_data = pd.read_csv("EDACaP040706_daily.csv")
historical_data['month_day'] = historical_data['month'].astype(str).str.zfill(2) + '-' + historical_data['day'].astype(str).str.zfill(2)

historical_averages = historical_data.groupby('month_day').agg({
    'prec': 'mean',      # Average daily precipitation
    't_max': 'mean',     # Average daily max temperature
    't_min': 'mean'      # Average daily min temperature
}).reset_index()

historical_averages.columns = ['month_day', 'avg_prec', 'avg_t_max', 'avg_t_min']
historical_averages.head()

# Fetch forecast data
forecast_data = fetch_weather_forecast(api_url)
anomalies_df = calculate_anomalies(forecast_data, historical_averages)

# Define the growth stage (e.g., "pre-planting", "early growth")
growth_stage = "pre-planting"

# Planting date advisory
planting_recommendations = planting_date_advisory(anomalies_df)
for recommendation in planting_recommendations:
    print(recommendation)

# Define scenarios for the selected growth stage
scenarios = define_scenarios(anomalies_df, growth_stage)

# Generate advisory using ChatGPT
advisory_report

KeyError: 'avg_rh'